
## Portfolio Optimization with Black-Litterman
## Multi-Benchmark Tracking Error Visualization



## Import Required Libraries

In [13]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize


## Download Historical Data


In [34]:

tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA"]
data = yf.download(tickers, start="2020-01-01", end="2024-01-01")["Close"]

[*********************100%***********************]  6 of 6 completed
[*********************100%***********************]  6 of 6 completed


In [35]:
data.head()

Ticker,AAPL,AMZN,GOOGL,META,MSFT,NVDA
Date,,,,,,
2020-01-02,72.538498,94.900497,67.965233,208.635391,152.791107,5.971410
2020-01-03,71.833313,93.748497,67.609688,207.531479,150.888596,5.875833
2020-01-06,72.405670,95.143997,69.411766,211.440018,151.278641,5.900473
2020-01-07,72.065147,95.343002,69.277687,211.897522,149.899307,5.971908
2020-01-08,73.224403,94.598503,69.770775,214.045731,152.286957,5.983109


# Compute log returns

In [36]:
returns = np.log(data / data.shift(1)).dropna()
mu = returns.mean().values * 252   # annualized mean returns
Sigma = returns.cov().values * 252 # annualized covariance

n_assets = len(tickers)

## Define Benchmarks

In [37]:
# Market-cap benchmark (using yfinance market cap at end date)
info = {t: yf.Ticker(t).info for t in tickers}
market_caps = np.array([info[t]["marketCap"] for t in tickers], dtype=float)
market_w = market_caps / market_caps.sum()

bench_eq = np.repeat(1.0 / n_assets, n_assets)
bench_tilt = np.roll(market_w, 2)

benchmarks = {
    "MarketCap": market_w,
    "EqualWeight": bench_eq,
    "Tilted": bench_tilt
}


## Black–Litterman Model


In [38]:
delta = 2.5
pi = delta * Sigma.dot(market_w)  # equilibrium implied returns

# One simple view: AAPL expected to outperform MSFT by 2%
P = np.zeros((1, n_assets))
P[0, tickers.index("AAPL")] = 1
P[0, tickers.index("MSFT")] = -1
q = np.array([0.02])

tau = 0.05
Omega = np.array([[P.dot(tau * Sigma).dot(P.T)[0,0]]])

inv_tauSigma = np.linalg.inv(tau * Sigma)
inv_Omega = np.linalg.inv(Omega)
middle = np.linalg.inv(inv_tauSigma + P.T.dot(inv_Omega).dot(P))
right = inv_tauSigma.dot(pi) + P.T.dot(inv_Omega).dot(q)
mu_bl = middle.dot(right).flatten()

## Efficient Frontier Function

In [39]:
def min_variance_target(mu, Sigma, target_return):
    n = len(mu)
    def obj(x): return 0.5 * x.dot(Sigma).dot(x)
    cons = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
        {'type': 'eq', 'fun': lambda x: x.dot(mu) - target_return}
    )
    x0 = np.repeat(1.0/n, n)
    res = minimize(obj, x0, constraints=cons, method='SLSQP')
    if not res.success: return None, None, None
    w = res.x
    var = w.dot(Sigma).dot(w)
    ret = w.dot(mu)
    return w, var, ret

def build_frontier(mu, Sigma, label):
    targets = np.linspace(mu.min(), mu.max(), 50)
    weights, vars_, rets = [], [], []
    for t in targets:
        w, v, r = min_variance_target(mu, Sigma, t)
        if w is not None:
            weights.append(w); vars_.append(v); rets.append(r)
    return {"weights": np.array(weights), "vars": np.array(vars_), "rets": np.array(rets)}

frontiers = {
    "Prior (Implied)": build_frontier(pi, Sigma, "Prior"),
    "Black-Litterman": build_frontier(mu_bl, Sigma, "BL")
}


## Tracking Error Function

In [40]:
def tracking_error(w, wb, Sigma):
    diff = w - wb
    return np.sqrt(diff.dot(Sigma).dot(diff))

te_results = {}
for label, data in frontiers.items():
    rets, ws = data["rets"], data["weights"]
    te_matrix = np.zeros((len(ws), len(benchmarks)))
    for i, w in enumerate(ws):
        for j, (bn_name, wb) in enumerate(benchmarks.items()):
            te_matrix[i,j] = tracking_error(w, wb, Sigma)
    te_results[label] = {"rets": rets, "te": te_matrix}

## Plots